## Objective
A proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system categorizes emails into product inquiries or order requests and generates appropriate email responses based on product catalog information and stock status.

### Inputs

The data is available in the repo. Originally written into a Google sheet.

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Functionalities

#### 1. Email classification
    
It classifies each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: **email-classification** sheet with columns: email ID, category.

#### 2. Order requests processing
1.   It processes order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, it creates a new order line with the status **created**. If the order cannot be fully fulfilled, it creates a line with the status **out of stock**. After placing the order, it updates the stock to accurately reflect the current inventory levels.

    **Output**: **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   It generates and saves response emails based on order processing results. Depending on the order status email informs the customer that their order was processed or could not be fulfilled. If the order were successfully processed, it sends an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, it sends an email explaining the situation and specifying which items are out of stock.

    **Output**: **order-response** sheet with columns: email ID, response.

#### 3. Product inquiry handling

Customers may ask general open questions.
- It responds to product inquiries using the information from the product catalog.


**Output**: **inquiry-response** sheet with columns: email ID, response.


# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0


In [ ]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(

    base_url='ChatGPT_API',

    api_key='YOUR_KEY_HERE'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

#Alternatively, you could use local data sheets
def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = 'YOUR_GOOGLE_DOCUMENT_ID'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [ ]:
def classify_emails(email_df, openai_client):
    classifications = []

    for _, row in email_df.iterrows():
        email_text = f"Subject: {row['subject']}\nMessage: {row['message']}"

        completion = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an email classifier for a fashion store. Classify emails into 'product inquiry' or 'order request' based on their content. Only return the classification results"},
                {"role": "user", "content": email_text}
            ]
        )

        classification = completion.choices[0].message.content.strip().lower()
        classifications.append({"email_id": row['email_id'], "category": classification})

    classification_df = pd.DataFrame(classifications)
    return classification_df

email_classification_df = classify_emails(emails_df, client)


# Task 2. Process order requests

In [ ]:

def process_orders(email_classification_df, emails_df, products_df, openai_client):
    order_status = []

    for _, row in email_classification_df[email_classification_df['category'] == 'order request'].iterrows():
        email_id = row['email_id']
        email_text = emails_df.loc[emails_df['email_id'] == email_id, 'message'].iloc[0]

        completion = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an order processor for a fashion store. Extract product details (product ID without whitespaces and quantity) from customer order emails. If the quantity is not explicitly mentioned, assume it's 1. If only the product name is mentioned, use it to try to match it to the products dataframe. If no match is found, output an empty list. If both product name and product if are not mentioned, ignore the email."},
                {"role": "user", "content": email_text},
                {"role": "system", "content": f"Available products:\n{products_df[['product_id', 'name']].to_markdown(index=False, numalign='left', stralign='left')}"}
            ]
        )

        try:
            order_details = eval(completion.choices[0].message.content)  # Convert string representation to list of dicts
        except (NameError, SyntaxError):  # Handle cases where the model doesn't output a valid list of dicts
            order_details = []

        print(order_details)
        for item in order_details:
            product_id = item.get('product_id')
            product_name = item.get('name')  # Handle cases where only product name is provided
            quantity = item.get('quantity', 1)  # Default to 1 if not specified

            # Find matching product using either product_id or product_name
            product = products_df.loc[(products_df['product_id'] == product_id) | (products_df['name'] == product_name)].iloc[0] if not products_df.loc[(products_df['product_id'] == product_id) | (products_df['name'] == product_name)].empty else None

            if product:
                if product['stock'] >= quantity:
                    order_status.append({"email_id": email_id, "product_id": product['product_id'], "quantity": quantity, "status": "created"})
                    product['stock'] -= quantity
                else:
                    order_status.append({"email_id": email_id, "product_id": product['product_id'], "quantity": quantity, "status": "out of stock"})
            else:
                # Handle cases where no product is found
                order_status.append({"email_id": email_id, "product_id": product_id, "quantity": quantity, "status": "product not found"})


    order_status_df = pd.DataFrame(order_status)
    return order_status_df, products_df


# Process orders and update products
order_status_df, updated_products_df = process_orders(email_classification_df, emails_df, products_df, client)



[]
[]
[]
[]


KeyboardInterrupt: 

In [ ]:
def generate_response_emails(order_status_df, emails_df, products_df, openai_client):
    responses = []

    for _, row in order_status_df.iterrows():
        email_id = row['email_id']
        product_id = row['product_id']
        quantity = row['quantity']
        status = row['status']

        recipient_email = emails_df.loc[emails_df['email_id'] == email_id, 'subject'].iloc[0] # Assuming the subject contains the recipient email
        product_name = products_df.loc[products_df['product_id'] == product_id, 'name'].iloc[0] if product_id else 'N/A'  # Handle cases where product_id is not found

        completion = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an email generator for a fashion store. Generate a response email based on order processing results."},
                {"role": "user", "content": f"Order status: {status}\nProduct ID: {product_id}\nProduct Name: {product_name}\nQuantity: {quantity}\nRecipient Email: {recipient_email}"}
            ]
        )
        response_text = completion.choices[0].message.content
        responses.append({"email_id": email_id, "response": response_text})

    response_df = pd.DataFrame(responses)
    return response_df


# ... (Assuming you have your order_status_df, emails_df, and products_df loaded)

# Initialize OpenAI client
client = OpenAI()

# Generate responses
response_df = generate_response_emails(order_status_df, emails_df, products_df, client)


# Task 3. Handle product inquiry
### TBD